In [1]:
import backtrader as bt
import pandas as pd
import datetime
import backtrader.feeds as btfeeds
from IPython.core.interactiveshell import InteractiveShell 
import pyecharts.options as opts
from pyecharts.charts import Grid,Line,Page,Scatter,Page,Bar
from pyecharts.faker import Faker
InteractiveShell.ast_node_interactivity = "all"
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
import numpy as np
from collections import deque
from tabulate import tabulate
st_date = datetime.datetime(2023, 5, 4)
ed_date = datetime.datetime(2023, 5, 20)

In [3]:
fef_codes = [
            'FEF2302','FEF2303','FEF2304','FEF2305','FEF2306'
            ]
i_codes = [
            'I2302','I2303','I2304','I2305','I2306'
            ]

fef_data = {}
i_data = {}

for a in i_codes:
    i_data[a] =  pd.read_excel('Data/I/'+a+'.xlsx', index_col = 0, skiprows = [0,1], parse_dates=True).dropna()
for a in fef_codes:
    fef_data[a] =  pd.read_excel('Data/FEF/'+a+'.xlsx', index_col = 0, skiprows = [0,1], parse_dates=True).dropna()

usdcnh = pd.read_csv('Data/USDCNH.csv', index_col = 0, skiprows = [0], parse_dates=True, encoding='ISO-8859-1')

In [4]:
i_feeds = []
fef_feeds = []

fef_rolldate = []
i_rolldate = []

for k in fef_data.keys():
    if fef_data[k].index[-1] < pd.to_datetime(st_date):
        # print(fef_data[k].index[-1])
        continue
    fef_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    fef_data[k]['Volume'] = fef_data[k]['Volume']*100000
    fef_data[k] = fef_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    fef_data[k] = fef_data[k].fillna(method = 'ffill')
    fef_data[k] = fef_data[k].loc[fef_data[k].index&usdcnh.index].dropna().sort_index().drop_duplicates()
    fef_data[k] = fef_data[k][fef_data[k].index.isin(usdcnh.index)]
    fef_feeds.append(bt.feeds.PandasData(dataname = fef_data[k], name = k))
    fef_rolldate.append(fef_data[k].resample('1d').last().index[-1]+datetime.timedelta(hours=14,minutes=57))



usdcnh = usdcnh.between_time('13:00','11:29').between_time('9:30','14:59').loc[st_date:].sort_index()
usdcnh.fillna(inplace = True, method = 'ffill')

for k in i_data.keys():    
    # i_data[k]
    if i_data[k].index[-1] < pd.to_datetime(st_date):
        # print(i_data[k].index[-1])
        continue
    i_data[k].columns = ['Open','High','Low','Close','Volume','OpenInterest']
    i_data[k]['Volume'] = i_data[k]['Volume']*100000
    i_data[k] = i_data[k].between_time('13:00','11:29').between_time('9:30','14:59')
    i_data[k] = i_data[k].fillna(method= 'ffill').sort_index().drop_duplicates()
    i_data[k] = i_data[k][i_data[k].index.isin(usdcnh.index)]
    i_feeds.append(bt.feeds.PandasData(dataname = i_data[k],name = k))
    i_rolldate.append(i_data[k].resample('1d').last().index[-2]+datetime.timedelta(hours=14, minutes=57))

usdcnh_feed = bt.feeds.PandasData(dataname = usdcnh, fromdate = st_date, todate = ed_date)

fef_rolldate = fef_rolldate[:-1]
i_rolldate = i_rolldate[:-1]

def fef_expire(dt,d):
    return dt in fef_rolldate

def i_expire(dt,d):
    if dt in i_rolldate:
        return True
    return False

C:\Users\siruo\AppData\Local\Temp\ipykernel_5228\861822869.py:15: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  fef_data[k] = fef_data[k].loc[fef_data[k].index&usdcnh.index].dropna().sort_index().drop_duplicates()
C:\Users\siruo\AppData\Local\Temp\ipykernel_5228\861822869.py:15: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  fef_data[k] = fef_data[k].loc[fef_data[k].index&usdcnh.index].dropna().sort_index().drop_duplicates()


In [5]:
class FEFIStrategy(bt.Strategy):
    params = (

        ('margin', 4),  
        ('distor', 1),
        ('bias', 0),
        ('i_lot',1),
        ('fef_lot',4),
        ('logname','log.csv'),

    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def write_record(self, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime()
        self.portlog.write('%s, %.1f, %.1f, %.1f, %d, %d, %d, %d\n' 
                                        % ( dt.isoformat(),
                                            self.usdcnh_close[0],
                                            self.i_close[0],
                                            self.fef_close[0],
                                            self.getpositionbyname('I').size,
                                            self.getpositionbyname('FEF').size,
                                            self.broker.get_cash(),
                                            self.broker.get_value()
                                       )
        )
    

    def notify_order(self, order):  
        if order.status in [order.Completed]:
            ops = None
            
            if order.isbuy():
                ops = 'Buy'
            elif order.issell():
                ops = 'Sell'
            if order.isbuy():
                self.log('BUY EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                        
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )

            else:  # Sell
                self.log('SELL EXECUTED,%s,%.2f,%d,%.2f,%d,%d,%s,%.2f\n' % ( 
                                                        order.data._name,
                                                       
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
            self.tradeLog.write('%s,%s,%s,%.2f,%d,%.2f,%d,%d,%d,%s,%.2f\n' % (
                                                        order.data._name,
                                                        bt.num2date(order.executed.dt),
                                                        ops,
                                                        order.executed.price,
                                                        order.executed.size,
                                                        order.executed.pnl,
                                                        self.base,
                                                        order.executed.psize,
                                                        order.executed.comm,
                                                        order.ref,
                                                        self.broker.get_value())
                                                    )
        self.order = None
        # Sentinel to None: new orders allowed
    def __init__(self):
        self.tradeLog = open('tradelog/'+self.p.logname, 'w+')
        self.tradeLog.write('Symbol,Datetime,Trade,Price,Size,PnL,Base,CurrentPos,Comm,ref,Value\n')
        self.portlog = open('portlog/'+self.p.logname, 'w+')
        self.portlog.write('Datetime,USDCNH,I,FEF,I_pos,FEF_pos,Cash,Value\n')
        self.order = None
        self.traded = False
        self.i_close = self.data0.close
        self.fef_close = self.data1.close
        self.usdcnh_close = self.data2.close
        self.fef_theo = None 
        self.theo_spread = None 
        self.lots_int = None
        self.base = 0
        
    def next(self):
        
        self.fef_theo = self.usdcnh_close[0]*self.i_close[0]
        # print(self.a50_close[0],self.csi300_close[0],self.csi500_close[0],self.if_close[0],self.ic_close[0])
        fef_bid = self.fef_theo - self.p.margin + self.base
        fef_ask = self.fef_theo + self.p.margin + self.base
        self.log("%.2f,%.2f,%.d,%.d"%(self.i_close[0],self.fef_close[0],self.usdcnh_close[0],self.broker.get_value()))
        self.write_record()
        if self.fef_close > fef_ask: # 市场价 大于 理论卖价，市场价过高， 按照市场价卖出FEF
            size = (self.fef_close - fef_ask)//self.p.distor + 1 # size > 0
            self.buy( data = self.getdatabyname('I'),
                      size = self.p.i_lot*size,
                      exectype=bt.Order.Market,coc=True)
            self.sell( data = self.getdatabyname('FEF'),
                       size = self.p.fef_lot*size, # size < 0
                       exectype=bt.Order.Market,coc=True)
            self.base = self.base + size * self.p.distor # base 上移，抬高理论价
            
        if self.fef_close < fef_bid: # 市场价 小于 理论买价，市场价过低，按照市场价买入
            size = (fef_bid - self.fef_close)//self.p.distor + 1 # size > 0
            self.sell( data = self.getdatabyname('I'),
                       size = self.p.i_lot*size,
                       exectype=bt.Order.Market,coc=True)
            self.buy( data = self.getdatabyname('FEF'),
                      size = self.p.fef_lot*size, # size < 0
                      exectype=bt.Order.Market,coc=True)
            self.base = self.base - size * self.p.distor

        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in i_rolldate:
            print('ROLL')
            self.base = 0

            if self.getpositionbyname('I').size != 0:
                self.order_target_size( data = self.getdatabyname('I'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('FEF').size != 0:
                self.order_target_size( data = self.getdatabyname('FEF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
        if self.data.datetime.datetime()+datetime.timedelta(minutes=1) in fef_rolldate:
            print('ROLL')
            self.base = 0
            if self.getpositionbyname('I').size != 0:
                self.order_target_size( data = self.getdatabyname('I'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)
            if self.getpositionbyname('FEF').size != 0:
                self.order_target_size( data = self.getdatabyname('FEF'),
                                        target= 0,
                                        exectype=bt.Order.Market,
                                        coc = True)

    def stop(self):
        self.tradeLog.close()
        self.portlog.close()

In [6]:
from tabulate import tabulate

cerebro = bt.Cerebro(cheat_on_open = True)
cerebro.broker.set_coc(True)
cerebro.broker.set_cash(1000000)
cerebro.broker.set_shortcash(False)

cerebro.rolloverdata(name='I', *i_feeds, checkdate = i_expire, fromdate=st_date, todate=ed_date)
cerebro.rolloverdata(name='FEF', *fef_feeds, checkdate = fef_expire, fromdate=st_date, todate=ed_date)
cerebro.adddata(usdcnh_feed)

cerebro.broker.setcommission(commission=2.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=25,
                             margin=4000,
                             name='I')
#cerebro.broker.setcommission(commission=30,
                             #commtype=bt.CommInfoBase.COMM_FIXED,
                             #mult=200,
                             #margin=160000,
                             #name='IC')
cerebro.broker.setcommission(commission=1.5,
                             commtype=bt.CommInfoBase.COMM_FIXED,
                             mult=1,
                             margin=1000,
                             name='FEF')


cerebro.addstrategy(FEFIStrategy, i_lot = 1, fef_lot = 4, logname = 'I_4FEF_Basis.csv')


strats = cerebro.run(runonce = False, cheat_on_open = True, tradehistory = True)

0

2023-05-04T09:34:00, 845.00,102.95,6,1000000
2023-05-04T09:34:00, 845.00,103.05,6,1000000
2023-05-04T09:34:00, 845.00,103.05,6,1000000
2023-05-04T09:34:00, 845.00,103.05,6,1000000
2023-05-04T09:38:00, 845.00,103.15,6,1000000
2023-05-04T09:38:00, 845.00,103.15,6,1000000
2023-05-04T09:38:00, 845.00,103.15,6,1000000
2023-05-04T09:41:00, 845.00,103.40,6,1000000
2023-05-04T09:41:00, SELL EXECUTED,I,845.00,-1,0.00,-1,2,3,999991.90

2023-05-04T09:41:00, BUY EXECUTED,FEF,103.40,4,0.00,4,6,4,999991.90

2023-05-04T09:41:00, 845.00,103.50,6,999991
2023-05-04T09:43:00, 845.00,103.50,6,999991
2023-05-04T09:44:00, 845.00,103.50,6,999991
2023-05-04T09:44:00, 845.00,103.50,6,999991
2023-05-04T09:44:00, SELL EXECUTED,I,845.00,-1,0.00,-2,2,5,999984.20

2023-05-04T09:44:00, BUY EXECUTED,FEF,103.50,4,0.00,8,6,6,999984.20

2023-05-04T09:44:00, 845.00,103.60,6,999984
2023-05-04T09:47:00, 844.50,103.50,6,1000008
2023-05-04T09:47:00, 844.50,103.60,6,1000009
2023-05-04T09:49:00, 845.00,103.50,6,999983
2023-05-

In [7]:
port_record_set = {}

port_record = pd.read_csv('portlog/I_4FEF_Basis.csv')
port_record = port_record[['Datetime','USDCNH','I','FEF','I_pos','FEF_pos','Cash','Value']]
port_record.set_index('Datetime',inplace=True)
port_record = port_record.dropna()
port_record.index = pd.to_datetime(port_record.index)

port_record_set['Basis'] = port_record

In [8]:
port_record_set['Basis']

,USDCNH,I,FEF,I_pos,FEF_pos,Cash,Value
Datetime,,,,,,,
2023-05-04 09:34:00,6.9,845.0,103.0,0,0,1000000,1000000
2023-05-04 09:34:00,6.9,845.0,103.0,0,0,1000000,1000000
2023-05-04 09:34:00,6.9,845.0,103.0,0,0,1000000,1000000
2023-05-04 09:34:00,6.9,845.0,103.0,0,0,1000000,1000000
2023-05-04 09:38:00,6.9,845.0,103.2,0,0,1000000,1000000
...,...,...,...,...,...,...,...
2023-05-19 14:55:00,7.0,813.0,107.2,-23,-68,1132583,1292583
2023-05-19 14:56:00,7.0,811.0,107.2,-23,-68,1133733,1293733
2023-05-19 14:57:00,7.0,810.5,107.2,-12,-112,1133790,1293790


In [9]:
# port_record = port_record.resample('5min').last().dropna()
totalPortValue = (
        Line(
                init_opts=opts.InitOpts(
                width="1440px",
                height="500px",
                animation_opts=opts.AnimationOpts(animation=False),
                )
        )
        .add_xaxis([a.isoformat(sep=' ') for a in port_record_set['Basis'].index.to_list()])
        .add_yaxis(
                    series_name="基差",
                    y_axis=port_record_set['Basis'].Value-port_record_set['Basis'].Value[0],
                    is_symbol_show = False,
                    label_opts=opts.LabelOpts(is_show=False),
                    linestyle_opts=opts.LineStyleOpts(width =1)
                    )
        .extend_axis(
                    yaxis=opts.AxisOpts(
                                name = 'FEF Pos',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                splitline_opts=opts.SplitLineOpts(  is_show=False,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                    )
        .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            title_opts = opts.TitleOpts('3I-27FEF',),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'Value',
                                max_ = 'dataMax',
                                min_ = 'dataMin',
                                axislabel_opts=opts.LabelOpts(formatter="{value} W"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.2),)
                            ),
                            datazoom_opts=[
                                opts.DataZoomOpts(
                                    is_show=True, type_="inside", xaxis_index=[0, 0],#filter_mode = 'weakFilter',
                                ),
                                opts.DataZoomOpts(
                                    is_show=True, type_="slider",xaxis_index=[0, 0], pos_top="93%",#filter_mode = 'weakFilter',
                                ),
                                # opts.DataZoomOpts(
                                #     is_show=True, type_="slider",orient= "vertical",yaxis_index=[0, 0,1],range_start= -10,range_end=110
                                # ),
                            ],
                             tooltip_opts=opts.TooltipOpts(
                                trigger="axis",
                                axis_pointer_type="cross",
                                background_color="rgba(245, 245, 245, 0.8)",
                                border_width=1,
                                border_color="#ccc",
                                textstyle_opts=opts.TextStyleOpts(color="#000"),
                            ),
                            axispointer_opts=opts.AxisPointerOpts(
                                is_show=True, link=[{"xAxisIndex": [0]},{"yAxisIndex": [0]},]
                            ),
                        )
    )
totalCashUsage = (
    Bar()
    .add_xaxis([a.isoformat(sep=' ') for a in port_record.index.to_list()])
    .add_yaxis(
                series_name="Cash",
                y_axis=port_record_set['Basis'].FEF_pos.values.tolist(),
                yaxis_index = 1,
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts = opts.ItemStyleOpts(opacity=0.15, )
    )
    .set_global_opts(
                            legend_opts = opts.LegendOpts(pos_top = '1%',pos_left = '43%'),
                            xaxis_opts=opts.AxisOpts(
                                axislabel_opts=opts.LabelOpts(is_show = True),
                            ),
                            yaxis_opts=opts.AxisOpts(
                                name = 'position',
                                # max_ = 200,
                                # min_ = 0,
                                axislabel_opts=opts.LabelOpts(formatter="{value} K"),
                                splitline_opts=opts.SplitLineOpts(  is_show=True,
                                                                    linestyle_opts=opts.LineStyleOpts(opacity = 0.5),)
                            ),

    )
)

totalPortValue.overlap(totalCashUsage)
totalPortValue.render('4fef1i.html')

'c:\\Users\\siruo\\Downloads\\main3\\4fef1i.html'